In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import MDAnalysis as md
import MDAnalysis.analysis.rms
import pandas as pd
import rpy2
from rpy2.robjects.packages import importr
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()
import pandas.rpy.common as com
import matplotlib.pyplot as plt
from itertools import cycle
import math
%matplotlib inline

/Users/melvrl13/anaconda/lib/python2.7/site-packages/ipykernel/pylab/config.py:66: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  inline backend."""
/Users/melvrl13/anaconda/lib/python2.7/site-packages/ipykernel/pylab/config.py:71: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  'retina', 'jpeg', 'svg', 'pdf'.""")
/Users/melvrl13/anaconda/lib/python2.7/site-packages/ipykernel/pylab/config.py:85: DeprecationWarning: metadata {'config': True} was set from the constructor.  Metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  use `figure_formats` instead)""")
/Users/melvrl13/anaconda/lib/python2.7/site-packages/ipykernel/pylab/config.py:95: DeprecationWarning: metadata {'config': True} was set from t

In [13]:
topology = '/Volumes/RyanMdata/F10/F10.pdb'
trajectory = '/Volumes/RyanMdata/F10/CaCl2_150mM/cat16/F10_150mMCaCl2.dcd'
u = md.Universe(topology,trajectory)
sel = u.select_atoms('all')

In [15]:
# Format trajectory for use in R
temp = u.trajectory.timeseries(sel,format=u'fac')
X = temp.reshape((16000,289*3))

In [ ]:
# Make a distance matrix for use in R
numframes = len(u.trajectory)
d = np.zeros((numframes, numframes))
for i in xrange(numframes):
    for j in xrange(i-1):
        d[i,j] = MDAnalysis.analysis.rms.rmsd( sel.coordinates(u.trajectory[i]), 
                                              sel.coordinates(u.trajectory[j]), 
                                              center=True)
        d[j,i] = d[i,j]

In [12]:
#All the things from R
# Prepare trajectory data for R
nr,nc = X.shape
Xr = ro.r.matrix(X, nrow=nr, ncol=nc)
ro.r.assign("X", Xr)

# Prepare distance matrix for R
nr,nc = d.shape
Dr = ro.r.matrix(d, nrow=nr, ncol=nc)
ro.r.assign("D", Dr)
ro.r('library(NbClust)')

#Test data with 30 clustering indices in R
max_clusters = int(round(math.sqrt(numframes)))
ro.r('nb <- NbClust(X, distance = "euclidean", min.nc = 2, max.nc = 20, method = "kmeans", index ="all")')
ro.r('nb')

RRuntimeError: Error in NbClust(X, distance = "euclidean", min.nc = 2, max.nc = 20, method = "kmeans",  : 
  The TSS matrix is indefinite. There must be too many missing values. The index cannot be calculated.
